# Advanced QAOA Techniques

## Learning Objectives
- Implement Recursive QAOA (RQAOA) for improved performance
- Learn Multi-Angle QAOA for better parameter optimization
- Understand warm-starting techniques
- Explore hardware-efficient QAOA variants
- Compare advanced methods on challenging problems

## Why Advanced QAOA?

Standard QAOA faces several challenges:
1. **Parameter optimization**: Exponentially hard landscape
2. **Circuit depth limitations**: NISQ device constraints
3. **Barren plateaus**: Gradients vanish for large systems
4. **Problem-specific performance**: No universal parameters

Advanced techniques address these limitations through:
- **Recursive approaches**: Break problems into smaller pieces
- **Adaptive parameters**: Learn from problem structure
- **Warm-starting**: Use classical solutions as initialization
- **Hardware optimization**: Reduce circuit complexity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional, Callable
import itertools
from dataclasses import dataclass
from scipy.optimize import minimize
import networkx as nx
from collections import defaultdict
import time

# Set random seed for reproducibility
np.random.seed(42)

print("Advanced QAOA Tutorial Environment Ready!")

## 1. Recursive QAOA (RQAOA)

RQAOA iteratively fixes variables and reduces problem size:

1. **Run QAOA** on full problem
2. **Identify confident bits** (high probability)
3. **Fix these variables** to their most likely values
4. **Create reduced problem** with fewer variables
5. **Repeat** until problem is small enough

**Benefits**:
- Reduces circuit depth requirements
- Often finds better solutions than standard QAOA
- Natural divide-and-conquer approach

In [ ]:
# First, let's import our basic QAOA implementation from the previous notebook

class QuantumState:
    """Quantum state representation for simulation"""
    
    def __init__(self, n_qubits: int):
        self.n_qubits = n_qubits
        self.n_states = 2**n_qubits
        # Initialize in uniform superposition |+>
        self.amplitudes = np.ones(self.n_states, dtype=complex) / np.sqrt(self.n_states)
    
    def apply_phase_gate(self, qubit_idx: int, phase: float):
        """Apply phase gate to specific qubit"""
        for state in range(self.n_states):
            if (state >> qubit_idx) & 1:  # If qubit is |1>
                self.amplitudes[state] *= np.exp(1j * phase)
    
    def apply_mixer(self, angle: float):
        """Apply mixer Hamiltonian (X rotations on all qubits)"""
        new_amplitudes = np.zeros_like(self.amplitudes)
        cos_half = np.cos(angle / 2)
        sin_half = -1j * np.sin(angle / 2)
        
        for state in range(self.n_states):
            # Apply X rotation to each qubit
            for qubit in range(self.n_qubits):
                flipped_state = state ^ (1 << qubit)
                new_amplitudes[state] += cos_half * self.amplitudes[state]
                new_amplitudes[flipped_state] += sin_half * self.amplitudes[state]
        
        # Normalize after applying to all qubits
        self.amplitudes = new_amplitudes / np.sqrt(self.n_qubits)
    
    def get_probabilities(self) -> np.ndarray:
        """Get measurement probabilities"""
        return np.abs(self.amplitudes)**2
    
    def sample(self, n_shots: int = 1000) -> List[int]:
        """Sample from the quantum state"""
        probabilities = self.get_probabilities()
        return np.random.choice(self.n_states, size=n_shots, p=probabilities)

class BasicQAOA:
    """Basic QAOA implementation for comparison"""
    
    def __init__(self, cost_hamiltonian: Callable[[int], float]):
        self.cost_hamiltonian = cost_hamiltonian
    
    def run_circuit(self, gammas: List[float], betas: List[float], 
                   n_qubits: int) -> QuantumState:
        """Run QAOA circuit"""
        state = QuantumState(n_qubits)
        
        # Apply QAOA layers
        for gamma, beta in zip(gammas, betas):
            # Cost Hamiltonian (phase separating)
            for basis_state in range(state.n_states):
                cost = self.cost_hamiltonian(basis_state)
                phase = gamma * cost
                state.amplitudes[basis_state] *= np.exp(-1j * phase)
            
            # Mixer Hamiltonian
            state.apply_mixer(2 * beta)
        
        return state
    
    def expectation_value(self, gammas: List[float], betas: List[float], 
                         n_qubits: int) -> float:
        """Calculate expectation value"""
        state = self.run_circuit(gammas, betas, n_qubits)
        probabilities = state.get_probabilities()
        
        expectation = 0
        for basis_state in range(state.n_states):
            cost = self.cost_hamiltonian(basis_state)
            expectation += probabilities[basis_state] * cost
        
        return expectation

print("Basic QAOA implementation ready!")

In [ ]:
class RecursiveQAOA:
    """Recursive QAOA implementation"""
    
    def __init__(self, 
                 original_problem: Dict,
                 confidence_threshold: float = 0.7,
                 max_recursion_depth: int = 5,
                 min_problem_size: int = 6):
        
        self.original_problem = original_problem
        self.confidence_threshold = confidence_threshold
        self.max_recursion_depth = max_recursion_depth
        self.min_problem_size = min_problem_size
        
        # Tracking
        self.recursion_history = []
        self.fixed_variables = {}
    
    def create_maxcut_cost_function(self, edges: List[Tuple[int, int]], 
                                   weights: List[float], n_qubits: int):
        """Create MaxCut cost function for current problem size"""
        def cost_function(bitstring_int: int) -> float:
            # Convert integer to binary array
            bitstring = [(bitstring_int >> i) & 1 for i in range(n_qubits)]
            
            cost = 0
            for (i, j), weight in zip(edges, weights):
                if i < n_qubits and j < n_qubits:  # Valid for current problem size
                    if bitstring[i] != bitstring[j]:
                        cost += weight
            
            return -cost  # Minimize negative (maximize cut)
        
        return cost_function
    
    def identify_confident_variables(self, state: QuantumState, 
                                   threshold: float) -> Dict[int, int]:
        """Identify variables with high confidence"""
        probabilities = state.get_probabilities()
        n_qubits = state.n_qubits
        
        # Calculate marginal probabilities for each qubit
        marginals = np.zeros((n_qubits, 2))  # [qubit][0 or 1]
        
        for basis_state in range(state.n_states):
            prob = probabilities[basis_state]
            for qubit in range(n_qubits):
                bit_value = (basis_state >> qubit) & 1
                marginals[qubit, bit_value] += prob
        
        # Find confident variables
        confident_vars = {}
        for qubit in range(n_qubits):
            max_prob = max(marginals[qubit])
            if max_prob >= threshold:
                confident_value = np.argmax(marginals[qubit])
                confident_vars[qubit] = confident_value
        
        return confident_vars
    
    def reduce_problem(self, edges: List[Tuple[int, int]], 
                      weights: List[float], 
                      fixed_vars: Dict[int, int]) -> Tuple[List[Tuple[int, int]], List[float], Dict[int, int]]:
        """Reduce problem by fixing variables"""
        # Create mapping from old to new variable indices
        remaining_vars = [i for i in range(max(max(edges, key=lambda x: max(x))) + 1) 
                         if i not in fixed_vars]
        var_mapping = {old_var: new_var for new_var, old_var in enumerate(remaining_vars)}
        
        # Filter and remap edges
        new_edges = []
        new_weights = []
        
        for (i, j), weight in zip(edges, weights):
            # Skip edges involving fixed variables
            if i in fixed_vars or j in fixed_vars:
                continue
            
            # Remap to new variable indices
            new_i = var_mapping[i]
            new_j = var_mapping[j]
            new_edges.append((new_i, new_j))
            new_weights.append(weight)
        
        return new_edges, new_weights, var_mapping
    
    def solve_recursive(self, 
                       edges: List[Tuple[int, int]], 
                       weights: List[float], 
                       depth: int = 0) -> Tuple[Dict[int, int], float]:
        """Recursively solve the problem"""
        
        if depth > self.max_recursion_depth:
            print(f"Max recursion depth {self.max_recursion_depth} reached")
            return {}, 0
        
        # Determine current problem size
        if not edges:
            return {}, 0
        
        n_qubits = max(max(edges, key=lambda x: max(x))) + 1
        
        print(f"\nDepth {depth}: Solving problem with {n_qubits} variables, {len(edges)} edges")
        
        # Base case: problem small enough for classical solver or brute force
        if n_qubits <= self.min_problem_size:
            print(f"Base case reached: solving {n_qubits}-variable problem classically")
            return self.solve_classically(edges, weights, n_qubits)
        
        # Run QAOA on current problem
        cost_function = self.create_maxcut_cost_function(edges, weights, n_qubits)
        qaoa = BasicQAOA(cost_function)
        
        # Optimize QAOA parameters (simplified)
        best_expectation = float('inf')
        best_state = None
        
        # Try different parameter combinations
        for p in [1, 2]:  # Circuit depth
            for trial in range(3):  # Multiple random starts
                gammas = np.random.uniform(0, np.pi, p)
                betas = np.random.uniform(0, np.pi/2, p)
                
                # Simple parameter optimization
                def objective(params):
                    g = params[:p]
                    b = params[p:]
                    return qaoa.expectation_value(g, b, n_qubits)
                
                result = minimize(objective, np.concatenate([gammas, betas]), 
                                method='COBYLA', 
                                options={'maxiter': 50, 'disp': False})
                
                if result.fun < best_expectation:
                    best_expectation = result.fun
                    opt_gammas = result.x[:p]
                    opt_betas = result.x[p:]
                    best_state = qaoa.run_circuit(opt_gammas, opt_betas, n_qubits)
        
        print(f"QAOA expectation value: {best_expectation:.3f}")
        
        # Identify confident variables
        confident_vars = self.identify_confident_variables(best_state, self.confidence_threshold)
        
        print(f"Found {len(confident_vars)} confident variables: {confident_vars}")
        
        if not confident_vars:
            print("No confident variables found, stopping recursion")
            # Return best solution from sampling
            samples = best_state.sample(1000)
            best_sample = min(samples, key=cost_function)
            solution = {i: (best_sample >> i) & 1 for i in range(n_qubits)}
            return solution, cost_function(best_sample)
        
        # Store this level's results
        self.recursion_history.append({
            'depth': depth,
            'n_qubits': n_qubits,
            'n_edges': len(edges),
            'confident_vars': confident_vars.copy(),
            'expectation': best_expectation
        })
        
        # Update global fixed variables
        for var, value in confident_vars.items():
            self.fixed_variables[var] = value
        
        # Reduce problem
        reduced_edges, reduced_weights, var_mapping = self.reduce_problem(
            edges, weights, confident_vars)
        
        if not reduced_edges:
            print("Problem fully reduced!")
            return confident_vars, best_expectation
        
        # Recursively solve reduced problem
        reduced_solution, reduced_cost = self.solve_recursive(
            reduced_edges, reduced_weights, depth + 1)
        
        # Combine solutions
        full_solution = confident_vars.copy()
        
        # Map reduced solution back to original variables
        reverse_mapping = {new: old for old, new in var_mapping.items()}
        for new_var, value in reduced_solution.items():
            if new_var in reverse_mapping:
                original_var = reverse_mapping[new_var]
                full_solution[original_var] = value
        
        return full_solution, reduced_cost
    
    def solve_classically(self, edges: List[Tuple[int, int]], 
                         weights: List[float], n_qubits: int) -> Tuple[Dict[int, int], float]:
        """Solve small problems classically"""
        cost_function = self.create_maxcut_cost_function(edges, weights, n_qubits)
        
        best_cost = float('inf')
        best_solution = {}
        
        # Brute force for small problems
        for bitstring_int in range(2**n_qubits):
            cost = cost_function(bitstring_int)
            if cost < best_cost:
                best_cost = cost
                best_solution = {i: (bitstring_int >> i) & 1 for i in range(n_qubits)}
        
        return best_solution, best_cost

print("Recursive QAOA implementation ready!")

In [ ]:
# Let's test RQAOA on a sample problem
def create_test_maxcut_problem(n_nodes: int = 8, edge_prob: float = 0.6) -> Tuple[List[Tuple[int, int]], List[float]]:
    """Create a random MaxCut problem for testing"""
    edges = []
    weights = []
    
    for i in range(n_nodes):
        for j in range(i + 1, n_nodes):
            if np.random.random() < edge_prob:
                edges.append((i, j))
                weights.append(np.random.uniform(0.5, 2.0))
    
    return edges, weights

# Create test problem
test_edges, test_weights = create_test_maxcut_problem(8, 0.4)
print(f"Test problem: {len(test_edges)} edges on 8 nodes")
print(f"Edges: {test_edges[:5]}...")

# Test RQAOA
rqaoa = RecursiveQAOA(
    original_problem={'edges': test_edges, 'weights': test_weights},
    confidence_threshold=0.65,
    max_recursion_depth=3,
    min_problem_size=4
)

print("\n=== Running Recursive QAOA ===")
start_time = time.time()
solution, cost = rqaoa.solve_recursive(test_edges, test_weights)
rqaoa_time = time.time() - start_time

print(f"\nRQAOA Solution: {solution}")
print(f"Cost: {cost:.3f}")
print(f"Time: {rqaoa_time:.2f}s")
print(f"Recursion levels: {len(rqaoa.recursion_history)}")

# Show recursion progression
for level in rqaoa.recursion_history:
    print(f"Level {level['depth']}: {level['n_qubits']} vars → {len(level['confident_vars'])} fixed")

## 2. Multi-Angle QAOA (MA-QAOA)

Multi-Angle QAOA allows different angles for different qubits/edges:
- **Standard QAOA**: Same γ and β for all gates
- **MA-QAOA**: Individual angles for each constraint/mixer

**Advantages**:
- More flexible parameter space
- Can capture problem structure better
- Often achieves higher approximation ratios

**Challenges**:
- Exponentially more parameters to optimize
- Risk of overfitting with limited quantum resources

In [ ]:
class MultiAngleQAOA:
    """Multi-Angle QAOA with individual parameters per constraint"""
    
    def __init__(self, edges: List[Tuple[int, int]], weights: List[float]):
        self.edges = edges
        self.weights = weights
        self.n_qubits = max(max(edges, key=lambda x: max(x))) + 1 if edges else 0
        self.n_edges = len(edges)
    
    def create_cost_function(self):
        """Create MaxCut cost function"""
        def cost_function(bitstring_int: int) -> float:
            bitstring = [(bitstring_int >> i) & 1 for i in range(self.n_qubits)]
            cost = 0
            for (i, j), weight in zip(self.edges, self.weights):
                if bitstring[i] != bitstring[j]:
                    cost += weight
            return -cost  # Minimize negative (maximize cut)
        return cost_function
    
    def run_circuit(self, gammas_per_edge: List[float], 
                   betas_per_qubit: List[float], layers: int = 1) -> QuantumState:
        """Run MA-QAOA circuit with individual angles"""
        state = QuantumState(self.n_qubits)
        
        # Reshape parameters for multiple layers
        gammas_per_layer = np.array(gammas_per_edge).reshape(layers, self.n_edges)
        betas_per_layer = np.array(betas_per_qubit).reshape(layers, self.n_qubits)
        
        for layer in range(layers):
            # Cost Hamiltonian with individual edge angles
            for basis_state in range(state.n_states):
                bitstring = [(basis_state >> i) & 1 for i in range(self.n_qubits)]
                total_phase = 0
                
                for edge_idx, ((i, j), weight) in enumerate(zip(self.edges, self.weights)):
                    if bitstring[i] != bitstring[j]:  # Edge is cut
                        total_phase += gammas_per_layer[layer, edge_idx] * weight
                
                state.amplitudes[basis_state] *= np.exp(-1j * total_phase)
            
            # Mixer Hamiltonian with individual qubit angles
            for qubit in range(self.n_qubits):
                beta = betas_per_layer[layer, qubit]
                # Apply individual X rotation to this qubit
                new_amplitudes = state.amplitudes.copy()
                cos_half = np.cos(beta)
                sin_half = -1j * np.sin(beta)
                
                for basis_state in range(state.n_states):
                    flipped_state = basis_state ^ (1 << qubit)
                    new_amplitudes[basis_state] = (
                        cos_half * state.amplitudes[basis_state] + 
                        sin_half * state.amplitudes[flipped_state]
                    )
                
                state.amplitudes = new_amplitudes
        
        return state
    
    def expectation_value(self, gammas_per_edge: List[float], 
                         betas_per_qubit: List[float], layers: int = 1) -> float:
        """Calculate expectation value for MA-QAOA"""
        state = self.run_circuit(gammas_per_edge, betas_per_qubit, layers)
        probabilities = state.get_probabilities()
        cost_function = self.create_cost_function()
        
        expectation = 0
        for basis_state in range(state.n_states):
            cost = cost_function(basis_state)
            expectation += probabilities[basis_state] * cost
        
        return expectation
    
    def optimize_parameters(self, layers: int = 1, max_iterations: int = 100) -> Dict:
        """Optimize MA-QAOA parameters"""
        # Initialize parameters
        n_gamma_params = self.n_edges * layers
        n_beta_params = self.n_qubits * layers
        
        # Random initialization
        initial_gammas = np.random.uniform(0, np.pi, n_gamma_params)
        initial_betas = np.random.uniform(0, np.pi/2, n_beta_params)
        initial_params = np.concatenate([initial_gammas, initial_betas])
        
        def objective(params):
            gammas = params[:n_gamma_params]
            betas = params[n_gamma_params:]
            return self.expectation_value(gammas, betas, layers)
        
        # Optimize
        result = minimize(objective, initial_params, 
                         method='COBYLA',
                         options={'maxiter': max_iterations, 'disp': False})
        
        optimal_gammas = result.x[:n_gamma_params]
        optimal_betas = result.x[n_gamma_params:]
        
        return {
            'optimal_gammas': optimal_gammas,
            'optimal_betas': optimal_betas,
            'optimal_value': result.fun,
            'success': result.success,
            'n_evaluations': result.nfev
        }

print("Multi-Angle QAOA implementation ready!")

## 3. Warm-Starting QAOA

Warm-starting initializes QAOA with classical solutions:

1. **Find classical solution** (greedy, simulated annealing, etc.)
2. **Initialize quantum state** to encode this solution
3. **Run short QAOA** to refine the solution

**Benefits**:
- Faster convergence
- Better final solutions
- Reduced parameter landscape complexity

In [ ]:
class WarmStartQAOA:
    """QAOA with warm-starting from classical solutions"""
    
    def __init__(self, edges: List[Tuple[int, int]], weights: List[float]):
        self.edges = edges
        self.weights = weights
        self.n_qubits = max(max(edges, key=lambda x: max(x))) + 1 if edges else 0
    
    def greedy_maxcut(self) -> List[int]:
        """Simple greedy algorithm for initial solution"""
        assignment = [-1] * self.n_qubits  # -1 means unassigned
        
        # Start with node 0 in partition 0
        assignment[0] = 0
        
        for node in range(1, self.n_qubits):
            # Calculate benefit of assigning to each partition
            benefit_0 = 0  # Benefit of assigning to partition 0
            benefit_1 = 0  # Benefit of assigning to partition 1
            
            for (i, j), weight in zip(self.edges, self.weights):
                if i == node and assignment[j] != -1:
                    if assignment[j] == 0:
                        benefit_1 += weight  # Different partitions
                    else:
                        benefit_0 += weight
                elif j == node and assignment[i] != -1:
                    if assignment[i] == 0:
                        benefit_1 += weight
                    else:
                        benefit_0 += weight
            
            # Assign to partition with higher benefit
            assignment[node] = 0 if benefit_0 >= benefit_1 else 1
        
        return assignment
    
    def create_warm_start_state(self, classical_solution: List[int], 
                               bias_strength: float = 2.0) -> QuantumState:
        """Create quantum state biased toward classical solution"""
        state = QuantumState(self.n_qubits)
        
        # Start with uniform superposition
        state.amplitudes = np.ones(state.n_states, dtype=complex) / np.sqrt(state.n_states)
        
        # Apply bias toward classical solution
        classical_bitstring = sum(bit * (2**i) for i, bit in enumerate(classical_solution))
        
        for basis_state in range(state.n_states):
            # Calculate Hamming distance from classical solution
            hamming_distance = bin(basis_state ^ classical_bitstring).count('1')
            
            # Apply exponential bias (closer states get higher amplitude)
            bias_factor = np.exp(-bias_strength * hamming_distance / self.n_qubits)
            state.amplitudes[basis_state] *= bias_factor
        
        # Renormalize
        norm = np.sqrt(np.sum(np.abs(state.amplitudes)**2))
        state.amplitudes /= norm
        
        return state
    
    def run_warm_qaoa(self, warm_start_solution: List[int], 
                     layers: int = 1, bias_strength: float = 2.0) -> Dict:
        """Run QAOA with warm starting"""
        # Create cost function
        def cost_function(bitstring_int: int) -> float:
            bitstring = [(bitstring_int >> i) & 1 for i in range(self.n_qubits)]
            cost = 0
            for (i, j), weight in zip(self.edges, self.weights):
                if bitstring[i] != bitstring[j]:
                    cost += weight
            return -cost
        
        # Initialize with warm start
        initial_state = self.create_warm_start_state(warm_start_solution, bias_strength)
        
        # Optimize QAOA parameters
        def objective(params):
            gammas = params[:layers]
            betas = params[layers:]
            
            # Start from warm state instead of uniform superposition
            state = QuantumState(self.n_qubits)
            state.amplitudes = initial_state.amplitudes.copy()
            
            # Apply QAOA layers
            for gamma, beta in zip(gammas, betas):
                # Cost Hamiltonian
                for basis_state in range(state.n_states):
                    cost = cost_function(basis_state)
                    phase = gamma * cost
                    state.amplitudes[basis_state] *= np.exp(-1j * phase)
                
                # Mixer Hamiltonian
                state.apply_mixer(2 * beta)
            
            # Calculate expectation value
            probabilities = state.get_probabilities()
            expectation = sum(prob * cost_function(i) 
                            for i, prob in enumerate(probabilities))
            return expectation
        
        # Optimize
        initial_params = np.random.uniform(0, np.pi/2, 2*layers)
        result = minimize(objective, initial_params, method='COBYLA',
                         options={'maxiter': 100, 'disp': False})
        
        return {
            'classical_solution': warm_start_solution,
            'classical_cost': -cost_function(sum(bit * (2**i) for i, bit in enumerate(warm_start_solution))),
            'optimal_params': result.x,
            'qaoa_cost': -result.fun,
            'improvement': -result.fun - (-cost_function(sum(bit * (2**i) for i, bit in enumerate(warm_start_solution)))),
            'success': result.success
        }

print("Warm-starting QAOA implementation ready!")

## 4. Comprehensive Benchmarking

Let's compare all QAOA variants on the same problems:

In [ ]:
def benchmark_qaoa_methods(edges: List[Tuple[int, int]], 
                          weights: List[float],
                          problem_name: str = "Test") -> Dict:
    """Comprehensive benchmark of all QAOA methods"""
    
    n_qubits = max(max(edges, key=lambda x: max(x))) + 1 if edges else 0
    results = {'problem': problem_name, 'n_qubits': n_qubits, 'n_edges': len(edges)}
    
    # Classical optimal (brute force for small problems)
    if n_qubits <= 12:
        print(f"Finding classical optimum for {n_qubits}-qubit problem...")
        def cost_function(bitstring_int: int) -> float:
            bitstring = [(bitstring_int >> i) & 1 for i in range(n_qubits)]
            cost = 0
            for (i, j), weight in zip(edges, weights):
                if bitstring[i] != bitstring[j]:
                    cost += weight
            return cost
        
        best_cost = 0
        best_solution = 0
        for bitstring_int in range(2**n_qubits):
            cost = cost_function(bitstring_int)
            if cost > best_cost:
                best_cost = cost
                best_solution = bitstring_int
        
        results['classical_optimal'] = {
            'cost': best_cost,
            'solution': [(best_solution >> i) & 1 for i in range(n_qubits)]
        }
        print(f"Classical optimum: {best_cost:.3f}")
    else:
        results['classical_optimal'] = None
        print("Problem too large for classical brute force")
    
    # 1. Standard QAOA
    print("\n=== Standard QAOA ===")
    start_time = time.time()
    
    def standard_cost_function(bitstring_int: int) -> float:
        bitstring = [(bitstring_int >> i) & 1 for i in range(n_qubits)]
        cost = 0
        for (i, j), weight in zip(edges, weights):
            if bitstring[i] != bitstring[j]:
                cost += weight
        return -cost  # Minimize negative
    
    standard_qaoa = BasicQAOA(standard_cost_function)
    
    # Simple parameter optimization for standard QAOA
    best_expectation = float('inf')
    best_solution_standard = None
    
    for layers in [1, 2]:
        for trial in range(3):
            initial_params = np.random.uniform(0, np.pi/2, 2*layers)
            
            def objective(params):
                gammas = params[:layers]
                betas = params[layers:]
                return standard_qaoa.expectation_value(gammas, betas, n_qubits)
            
            result = minimize(objective, initial_params, method='COBYLA',
                             options={'maxiter': 50, 'disp': False})
            
            if result.fun < best_expectation:
                best_expectation = result.fun
                opt_gammas = result.x[:layers]
                opt_betas = result.x[layers:]
                final_state = standard_qaoa.run_circuit(opt_gammas, opt_betas, n_qubits)
                samples = final_state.sample(1000)
                best_sample = min(samples, key=standard_cost_function)
                best_solution_standard = [(best_sample >> i) & 1 for i in range(n_qubits)]
    
    standard_time = time.time() - start_time
    results['standard_qaoa'] = {
        'cost': -best_expectation,
        'solution': best_solution_standard,
        'time': standard_time
    }
    print(f"Standard QAOA: {-best_expectation:.3f} (time: {standard_time:.2f}s)")
    
    # 2. Multi-Angle QAOA
    print("\n=== Multi-Angle QAOA ===")
    start_time = time.time()
    
    ma_qaoa = MultiAngleQAOA(edges, weights)
    ma_result = ma_qaoa.optimize_parameters(layers=1, max_iterations=50)
    
    # Get best solution from MA-QAOA
    final_state_ma = ma_qaoa.run_circuit(ma_result['optimal_gammas'], 
                                        ma_result['optimal_betas'], 1)
    samples_ma = final_state_ma.sample(1000)
    best_sample_ma = min(samples_ma, key=lambda x: ma_qaoa.create_cost_function()(x))
    best_solution_ma = [(best_sample_ma >> i) & 1 for i in range(n_qubits)]
    
    ma_time = time.time() - start_time
    results['multi_angle_qaoa'] = {
        'cost': -ma_result['optimal_value'],
        'solution': best_solution_ma,
        'time': ma_time,
        'n_parameters': len(ma_result['optimal_gammas']) + len(ma_result['optimal_betas'])
    }
    print(f"MA-QAOA: {-ma_result['optimal_value']:.3f} (time: {ma_time:.2f}s, params: {results['multi_angle_qaoa']['n_parameters']})")
    
    # 3. Warm-Start QAOA
    print("\n=== Warm-Start QAOA ===")
    start_time = time.time()
    
    ws_qaoa = WarmStartQAOA(edges, weights)
    classical_init = ws_qaoa.greedy_maxcut()
    ws_result = ws_qaoa.run_warm_qaoa(classical_init, layers=1)
    
    ws_time = time.time() - start_time
    results['warm_start_qaoa'] = {
        'classical_cost': ws_result['classical_cost'],
        'qaoa_cost': ws_result['qaoa_cost'],
        'improvement': ws_result['improvement'],
        'time': ws_time
    }
    print(f"Warm-Start: {ws_result['classical_cost']:.3f} → {ws_result['qaoa_cost']:.3f} "
          f"(+{ws_result['improvement']:.3f}, time: {ws_time:.2f}s)")
    
    # 4. Recursive QAOA (if problem is large enough)
    if n_qubits >= 6:
        print("\n=== Recursive QAOA ===")
        start_time = time.time()
        
        rqaoa = RecursiveQAOA(
            original_problem={'edges': edges, 'weights': weights},
            confidence_threshold=0.6,
            max_recursion_depth=2,
            min_problem_size=4
        )
        
        rqaoa_solution, rqaoa_cost = rqaoa.solve_recursive(edges, weights)
        rqaoa_time = time.time() - start_time
        
        # Calculate actual cost of RQAOA solution
        actual_cost = 0
        for (i, j), weight in zip(edges, weights):
            if i in rqaoa_solution and j in rqaoa_solution:
                if rqaoa_solution[i] != rqaoa_solution[j]:
                    actual_cost += weight
        
        results['recursive_qaoa'] = {
            'cost': actual_cost,
            'solution': rqaoa_solution,
            'time': rqaoa_time,
            'recursion_levels': len(rqaoa.recursion_history)
        }
        print(f"RQAOA: {actual_cost:.3f} (time: {rqaoa_time:.2f}s, levels: {len(rqaoa.recursion_history)})")
    
    return results

# Run comprehensive benchmark
print("\n" + "="*60)
print("COMPREHENSIVE QAOA BENCHMARK")
print("="*60)

# Test on medium-sized problem
test_edges, test_weights = create_test_maxcut_problem(8, 0.4)
benchmark_results = benchmark_qaoa_methods(test_edges, test_weights, "8-node Random")

# Print summary
print("\n" + "="*40)
print("BENCHMARK SUMMARY")
print("="*40)

if benchmark_results['classical_optimal']:
    optimal_cost = benchmark_results['classical_optimal']['cost']
    print(f"Classical Optimal: {optimal_cost:.3f}")
    print("\nApproximation Ratios:")
    
    methods = ['standard_qaoa', 'multi_angle_qaoa', 'warm_start_qaoa', 'recursive_qaoa']
    method_names = ['Standard QAOA', 'Multi-Angle QAOA', 'Warm-Start QAOA', 'Recursive QAOA']
    
    for method, name in zip(methods, method_names):
        if method in benchmark_results:
            if method == 'warm_start_qaoa':
                cost = benchmark_results[method]['qaoa_cost']
            else:
                cost = benchmark_results[method]['cost']
            ratio = cost / optimal_cost if optimal_cost > 0 else 0
            time_taken = benchmark_results[method]['time']
            print(f"  {name}: {ratio:.3f} (cost: {cost:.3f}, time: {time_taken:.2f}s)")

print("\nTime Comparison:")
for method, name in zip(methods, method_names):
    if method in benchmark_results:
        time_taken = benchmark_results[method]['time']
        print(f"  {name}: {time_taken:.2f}s")

In [ ]:
# Visualize benchmark results
plt.figure(figsize=(15, 10))

# Plot 1: Cost comparison
plt.subplot(2, 3, 1)
methods = []
costs = []
colors = ['blue', 'green', 'orange', 'red', 'purple']

if benchmark_results['classical_optimal']:
    methods.append('Classical\nOptimal')
    costs.append(benchmark_results['classical_optimal']['cost'])

if 'standard_qaoa' in benchmark_results:
    methods.append('Standard\nQAOA')
    costs.append(benchmark_results['standard_qaoa']['cost'])

if 'multi_angle_qaoa' in benchmark_results:
    methods.append('Multi-Angle\nQAOA')
    costs.append(benchmark_results['multi_angle_qaoa']['cost'])

if 'warm_start_qaoa' in benchmark_results:
    methods.append('Warm-Start\nQAOA')
    costs.append(benchmark_results['warm_start_qaoa']['qaoa_cost'])

if 'recursive_qaoa' in benchmark_results:
    methods.append('Recursive\nQAOA')
    costs.append(benchmark_results['recursive_qaoa']['cost'])

bars = plt.bar(methods, costs, color=colors[:len(methods)])
plt.title('Cost Comparison')
plt.ylabel('MaxCut Value')
plt.xticks(rotation=45)

# Add value labels on bars
for bar, cost in zip(bars, costs):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{cost:.2f}', ha='center', va='bottom')

# Plot 2: Time comparison
plt.subplot(2, 3, 2)
time_methods = []
times = []

if 'standard_qaoa' in benchmark_results:
    time_methods.append('Standard\nQAOA')
    times.append(benchmark_results['standard_qaoa']['time'])

if 'multi_angle_qaoa' in benchmark_results:
    time_methods.append('Multi-Angle\nQAOA')
    times.append(benchmark_results['multi_angle_qaoa']['time'])

if 'warm_start_qaoa' in benchmark_results:
    time_methods.append('Warm-Start\nQAOA')
    times.append(benchmark_results['warm_start_qaoa']['time'])

if 'recursive_qaoa' in benchmark_results:
    time_methods.append('Recursive\nQAOA')
    times.append(benchmark_results['recursive_qaoa']['time'])

bars = plt.bar(time_methods, times, color=colors[1:len(time_methods)+1])
plt.title('Execution Time Comparison')
plt.ylabel('Time (seconds)')
plt.xticks(rotation=45)

# Add value labels
for bar, time_val in zip(bars, times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{time_val:.2f}s', ha='center', va='bottom')

# Plot 3: Approximation ratios (if optimal is known)
if benchmark_results['classical_optimal']:
    plt.subplot(2, 3, 3)
    optimal_cost = benchmark_results['classical_optimal']['cost']
    
    qaoa_methods = []
    ratios = []
    
    if 'standard_qaoa' in benchmark_results:
        qaoa_methods.append('Standard\nQAOA')
        ratios.append(benchmark_results['standard_qaoa']['cost'] / optimal_cost)
    
    if 'multi_angle_qaoa' in benchmark_results:
        qaoa_methods.append('Multi-Angle\nQAOA')
        ratios.append(benchmark_results['multi_angle_qaoa']['cost'] / optimal_cost)
    
    if 'warm_start_qaoa' in benchmark_results:
        qaoa_methods.append('Warm-Start\nQAOA')
        ratios.append(benchmark_results['warm_start_qaoa']['qaoa_cost'] / optimal_cost)
    
    if 'recursive_qaoa' in benchmark_results:
        qaoa_methods.append('Recursive\nQAOA')
        ratios.append(benchmark_results['recursive_qaoa']['cost'] / optimal_cost)
    
    bars = plt.bar(qaoa_methods, ratios, color=colors[1:len(qaoa_methods)+1])
    plt.title('Approximation Ratios')
    plt.ylabel('Ratio to Optimal')
    plt.xticks(rotation=45)
    plt.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='Optimal')
    
    # Add value labels
    for bar, ratio in zip(bars, ratios):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                 f'{ratio:.3f}', ha='center', va='bottom')

# Plot 4: Parameter scaling
plt.subplot(2, 3, 4)
if 'multi_angle_qaoa' in benchmark_results:
    n_params_standard = 2  # gamma and beta for 1 layer
    n_params_ma = benchmark_results['multi_angle_qaoa']['n_parameters']
    
    plt.bar(['Standard QAOA', 'Multi-Angle QAOA'], 
            [n_params_standard, n_params_ma], 
            color=['blue', 'green'])
    plt.title('Number of Parameters')
    plt.ylabel('Parameter Count')
    
    for i, val in enumerate([n_params_standard, n_params_ma]):
        plt.text(i, val + 0.5, str(val), ha='center', va='bottom')

# Plot 5: Warm-start improvement
if 'warm_start_qaoa' in benchmark_results:
    plt.subplot(2, 3, 5)
    classical_cost = benchmark_results['warm_start_qaoa']['classical_cost']
    qaoa_cost = benchmark_results['warm_start_qaoa']['qaoa_cost']
    
    plt.bar(['Classical\nInitialization', 'After QAOA\nRefinement'], 
            [classical_cost, qaoa_cost], 
            color=['orange', 'red'])
    plt.title('Warm-Start Improvement')
    plt.ylabel('MaxCut Value')
    
    # Show improvement
    improvement = qaoa_cost - classical_cost
    plt.annotate(f'+{improvement:.3f}', 
                xy=(1, qaoa_cost), xytext=(1, qaoa_cost + 0.1),
                arrowprops=dict(arrowstyle='->', color='black'),
                ha='center', fontweight='bold')

# Plot 6: Problem structure visualization
plt.subplot(2, 3, 6)
# Create a simple graph visualization
G = nx.Graph()
for (i, j), weight in zip(test_edges, test_weights):
    G.add_edge(i, j, weight=weight)

pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=500, font_size=10, font_weight='bold')

# Draw edge weights
edge_labels = {(i, j): f'{w:.1f}' for (i, j), w in zip(test_edges, test_weights)}
nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=8)

plt.title(f'Problem Graph\n({len(G.nodes)} nodes, {len(G.edges)} edges)')
plt.axis('off')

plt.tight_layout()
plt.show()

print("\nVisualization complete!")

## 🎯 Practical Exercises

### Exercise 1: Parameter Sensitivity Analysis
Analyze how sensitive each QAOA variant is to parameter initialization.

### Exercise 2: Scaling Behavior
Study how performance changes with problem size for each method.

### Exercise 3: Problem Structure Impact
Test on different graph types (complete, sparse, regular) to understand when each method excels.

### Exercise 4: Hybrid Optimization
Combine multiple methods (e.g., warm-start + recursive QAOA).

In [ ]:
# Exercise 1: Parameter Sensitivity Analysis
def parameter_sensitivity_analysis(edges, weights, n_trials=10):
    """Analyze parameter sensitivity for different QAOA variants"""
    print("=== Parameter Sensitivity Analysis ===")
    
    # Test standard QAOA with different initializations
    def test_cost_function(bitstring_int: int) -> float:
        n_qubits = max(max(edges, key=lambda x: max(x))) + 1
        bitstring = [(bitstring_int >> i) & 1 for i in range(n_qubits)]
        cost = 0
        for (i, j), weight in zip(edges, weights):
            if bitstring[i] != bitstring[j]:
                cost += weight
        return -cost
    
    qaoa = BasicQAOA(test_cost_function)
    n_qubits = max(max(edges, key=lambda x: max(x))) + 1
    
    standard_results = []
    for trial in range(n_trials):
        # Random initialization
        gamma = np.random.uniform(0, np.pi)
        beta = np.random.uniform(0, np.pi/2)
        
        def objective(params):
            return qaoa.expectation_value([params[0]], [params[1]], n_qubits)
        
        result = minimize(objective, [gamma, beta], method='COBYLA',
                         options={'maxiter': 30, 'disp': False})
        standard_results.append(-result.fun)
    
    print(f"Standard QAOA (n={n_trials} trials):")
    print(f"  Mean: {np.mean(standard_results):.3f} ± {np.std(standard_results):.3f}")
    print(f"  Range: [{np.min(standard_results):.3f}, {np.max(standard_results):.3f}]")
    
    # Test Multi-Angle QAOA sensitivity
    ma_qaoa = MultiAngleQAOA(edges, weights)
    ma_results = []
    
    for trial in range(n_trials):
        result = ma_qaoa.optimize_parameters(layers=1, max_iterations=30)
        ma_results.append(-result['optimal_value'])
    
    print(f"\nMulti-Angle QAOA (n={n_trials} trials):")
    print(f"  Mean: {np.mean(ma_results):.3f} ± {np.std(ma_results):.3f}")
    print(f"  Range: [{np.min(ma_results):.3f}, {np.max(ma_results):.3f}]")
    
    # Visualization
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.hist(standard_results, bins=5, alpha=0.7, label='Standard QAOA')
    plt.xlabel('MaxCut Value')
    plt.ylabel('Frequency')
    plt.title('Standard QAOA Parameter Sensitivity')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.hist(ma_results, bins=5, alpha=0.7, label='Multi-Angle QAOA', color='green')
    plt.xlabel('MaxCut Value')
    plt.ylabel('Frequency')
    plt.title('Multi-Angle QAOA Parameter Sensitivity')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    return {
        'standard': {'mean': np.mean(standard_results), 'std': np.std(standard_results)},
        'multi_angle': {'mean': np.mean(ma_results), 'std': np.std(ma_results)}
    }

# Exercise 2: Scaling Behavior
def scaling_analysis(max_nodes=10):
    """Analyze how methods scale with problem size"""
    print("\n=== Scaling Behavior Analysis ===")
    
    node_counts = range(4, max_nodes + 1, 2)
    methods_data = {
        'standard_qaoa': {'costs': [], 'times': []},
        'multi_angle_qaoa': {'costs': [], 'times': []},
        'warm_start_qaoa': {'costs': [], 'times': []}
    }
    
    for n_nodes in node_counts:
        print(f"\nTesting {n_nodes} nodes...")
        edges, weights = create_test_maxcut_problem(n_nodes, 0.5)
        
        # Standard QAOA
        start_time = time.time()
        def cost_func(bitstring_int: int) -> float:
            bitstring = [(bitstring_int >> i) & 1 for i in range(n_nodes)]
            cost = 0
            for (i, j), weight in zip(edges, weights):
                if bitstring[i] != bitstring[j]:
                    cost += weight
            return -cost
        
        qaoa = BasicQAOA(cost_func)
        result = minimize(lambda params: qaoa.expectation_value([params[0]], [params[1]], n_nodes),
                         [np.pi/4, np.pi/8], method='COBYLA',
                         options={'maxiter': 20, 'disp': False})
        standard_time = time.time() - start_time
        
        methods_data['standard_qaoa']['costs'].append(-result.fun)
        methods_data['standard_qaoa']['times'].append(standard_time)
        
        # Multi-Angle QAOA
        start_time = time.time()
        ma_qaoa = MultiAngleQAOA(edges, weights)
        ma_result = ma_qaoa.optimize_parameters(layers=1, max_iterations=20)
        ma_time = time.time() - start_time
        
        methods_data['multi_angle_qaoa']['costs'].append(-ma_result['optimal_value'])
        methods_data['multi_angle_qaoa']['times'].append(ma_time)
        
        # Warm-Start QAOA
        start_time = time.time()
        ws_qaoa = WarmStartQAOA(edges, weights)
        classical_init = ws_qaoa.greedy_maxcut()
        ws_result = ws_qaoa.run_warm_qaoa(classical_init, layers=1)
        ws_time = time.time() - start_time
        
        methods_data['warm_start_qaoa']['costs'].append(ws_result['qaoa_cost'])
        methods_data['warm_start_qaoa']['times'].append(ws_time)
    
    # Visualization
    plt.figure(figsize=(12, 5))
    
    # Performance scaling
    plt.subplot(1, 2, 1)
    colors = ['blue', 'green', 'orange']
    for i, (method, data) in enumerate(methods_data.items()):
        method_name = method.replace('_', ' ').title()
        plt.plot(node_counts, data['costs'], 'o-', color=colors[i], label=method_name)
    
    plt.xlabel('Number of Nodes')
    plt.ylabel('MaxCut Value')
    plt.title('Performance vs Problem Size')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Time scaling
    plt.subplot(1, 2, 2)
    for i, (method, data) in enumerate(methods_data.items()):
        method_name = method.replace('_', ' ').title()
        plt.plot(node_counts, data['times'], 's-', color=colors[i], label=method_name)
    
    plt.xlabel('Number of Nodes')
    plt.ylabel('Execution Time (s)')
    plt.title('Runtime vs Problem Size')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return methods_data

# Run exercises
print("\n" + "="*50)
print("RUNNING EXERCISES")
print("="*50)

# Exercise 1
sensitivity_results = parameter_sensitivity_analysis(test_edges, test_weights, n_trials=8)

# Exercise 2  
scaling_results = scaling_analysis(max_nodes=8)

print("\n✅ All exercises completed!")
print("\n📝 Key Insights:")
print(f"1. Parameter sensitivity: Multi-Angle QAOA std = {sensitivity_results['multi_angle']['std']:.3f} vs Standard = {sensitivity_results['standard']['std']:.3f}")
print("2. Scaling: Multi-Angle QAOA requires more parameters but often achieves better performance")
print("3. Warm-starting provides consistent improvements over classical initialization")
print("4. Recursive QAOA is most beneficial for larger, structured problems")

## 📋 Summary

### What We've Learned

1. **Recursive QAOA (RQAOA)**:
   - Breaks large problems into smaller pieces
   - Often finds better solutions than standard QAOA
   - Natural for divide-and-conquer problems

2. **Multi-Angle QAOA**:
   - Individual parameters for each constraint/qubit
   - More flexible but exponentially more parameters
   - Can capture problem structure better

3. **Warm-Starting**:
   - Initialize with classical solutions
   - Faster convergence and better final solutions
   - Combines classical intuition with quantum enhancement

### Performance Trade-offs

| Method | Pros | Cons |
|--------|------|------|
| Standard QAOA | Simple, few parameters | Limited expressivity |
| Multi-Angle QAOA | High flexibility | Parameter explosion |
| Warm-Start QAOA | Fast convergence | Depends on classical quality |
| Recursive QAOA | Scales well | Requires confident variable identification |

### When to Use Each Method

- **Standard QAOA**: Small problems, initial exploration
- **Multi-Angle QAOA**: Medium problems with structure
- **Warm-Start QAOA**: When good classical heuristics exist
- **Recursive QAOA**: Large problems with clear decomposition

## 🎯 Next Steps

1. **Hybrid Classical-Quantum Methods**: Combine the best of both worlds
2. **Hardware-Efficient QAOA**: Adapt to specific quantum device constraints
3. **Problem-Specific Optimizations**: Tailor methods to specific optimization problems
4. **Error Mitigation**: Handle noise in NISQ devices

### 🔬 Advanced Topics to Explore

- Quantum error correction in optimization
- Barren plateau mitigation strategies
- Parameter initialization strategies
- Problem decomposition techniques
- Quantum advantage analysis

---

**Continue to the next notebook**: `../06_Hybrid_Approaches/01_Classical_Quantum_Integration.ipynb`